In [4]:
import numpy as np
import pandas as pd

In [5]:
train = pd.read_csv('train.csv',parse_dates=['Datetime'])
test = pd.read_csv('test.csv',parse_dates=['Datetime'])

In [6]:
train.head()

,ID,Item_ID,Datetime,Category_3,Category_2,Category_1,Price,Number_Of_Sales
0,30495_20140101,30495,2014-01-01,0,2.0,90,165.123,1
1,30375_20140101,30375,2014-01-01,0,2.0,307,68.666,5
2,30011_20140101,30011,2014-01-01,0,3.0,67,253.314,2
3,30864_20140101,30864,2014-01-01,0,2.0,315,223.122,1
4,30780_20140101,30780,2014-01-01,1,2.0,132,28.750,1


In [7]:
item_count_train = train.Item_ID.value_counts().to_dict()
item_count_test = test.Item_ID.value_counts().to_dict()

train['item_count'] = train['Item_ID'].map(item_count_train)
test['item_count'] = test['Item_ID'].map(item_count_test)

train['dayofweek'] = train.Datetime.dt.dayofweek
test['dayofweek'] = test.Datetime.dt.dayofweek

train['weekyear'] = train.Datetime.dt.weekofyear
test['weekyear'] = test.Datetime.dt.weekofyear

train['dayofmonth'] = train.Datetime.dt.day
test['dayofmonth'] = test.Datetime.dt.day

#train['is_quarter_start'] = train.Datetime.dt.is_quarter_start
#test['is_quarter_start'] = test.Datetime.dt.is_quarter_start

#train['is_quarter_start'] = train.is_quarter_start.astype(int)
#test['is_quarter_start'] = test.is_quarter_start.astype(int)

train['month'] = train.Datetime.dt.month
test['month'] = test.Datetime.dt.month

In [8]:
week_year_price = train.groupby(['Item_ID','weekyear'])['Price'].aggregate(np.median).reset_index()
week_year_price.rename(columns={'Price':'weekYearIDPrice'},inplace=True)

week_year_sales = train.groupby(['Item_ID','weekyear'])['Number_Of_Sales'].aggregate(np.median).reset_index()
week_year_sales.rename(columns={'Number_Of_Sales':'weekYearIDSales'},inplace=True)

train = pd.merge(train, week_year_price, on=['Item_ID','weekyear'], how='left')
train = pd.merge(train, week_year_sales, on=['Item_ID','weekyear'], how='left')

test = pd.merge(test, week_year_price, on=['Item_ID','weekyear'], how='left')
test = pd.merge(test, week_year_sales, on=['Item_ID','weekyear'], how='left')

In [9]:
train = train.loc[train['Number_Of_Sales'] <= 340000]

In [10]:
train.fillna(-1, inplace=True)
test.fillna(-1, inplace=True)

In [11]:
train.head()

,ID,Item_ID,Datetime,Category_3,Category_2,Category_1,Price,Number_Of_Sales,item_count,dayofweek,weekyear,dayofmonth,month,weekYearIDPrice,weekYearIDSales
0,30495_20140101,30495,2014-01-01,0,2.0,90,165.123,1,908,2,1,1,1,147.0690,12.0
1,30375_20140101,30375,2014-01-01,0,2.0,307,68.666,5,911,2,1,1,1,48.2630,31.5
2,30011_20140101,30011,2014-01-01,0,3.0,67,253.314,2,849,2,1,1,1,302.0160,3.0
3,30864_20140101,30864,2014-01-01,0,2.0,315,223.122,1,907,2,1,1,1,142.3630,15.0
4,30780_20140101,30780,2014-01-01,1,2.0,132,28.750,1,623,2,1,1,1,65.4005,2.0


In [14]:
for x in ['weekYearIDPrice','weekYearIDSales']:
    train[x] = np.log(train[x] + 10)
    test[x] = np.log(test[x]+10)

In [15]:
cols_to_use = ['Category_3','Category_2','Category_1','dayofweek','weekyear',\
              'dayofmonth','month','item_count','weekYearIDPrice','weekYearIDSales']

### validation data

In [16]:
train_X = train.loc[(train['Datetime'] < "2016-01-01") & (train['Datetime'] >= "2015-01-01")]
valid_X = train.loc[train['Datetime'] >= '2016-01-01']

In [17]:
import xgboost as xgb

/home/manish/anaconda2/envs/package15/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [18]:
dtrain1 = xgb.DMatrix(data=train_X[cols_to_use], label = np.log1p(train_X['Price']))
dvalid1 = xgb.DMatrix(data=valid_X[cols_to_use], label = np.log1p(valid_X['Price']))

dtrain2 = xgb.DMatrix(data=train_X[cols_to_use], label = np.log1p(train_X['Number_Of_Sales']))
dvalid2 = xgb.DMatrix(data=valid_X[cols_to_use], label = np.log1p(valid_X['Number_Of_Sales']))

In [19]:
dtest = xgb.DMatrix(data=test[cols_to_use])

In [20]:
xgb_params1 = {
    'colsample_bytree': 1,
    'subsample': 1,
    'eta': 0.1,
    'objective': 'reg:linear',
    'max_depth': 6,
    'min_child_weight': 10,
    'eval_metric': 'rmse',
    'alpha':8
}

xgb_params2 = {
    'colsample_bytree': 1,
    'subsample': 1,
    'eta': 0.1,
    'objective': 'reg:linear',
    'max_depth': 6,
    'min_child_weight': 0,
    'eval_metric': 'rmse',
    'alpha':8
}
watchlist1  = [ (dtrain1,'train'),(dvalid1,'valid')]
watchlist2  = [ (dtrain2,'train'),(dvalid2,'valid')]


In [21]:
bst1 = xgb.train(xgb_params1, dtrain1, 1000, watchlist1, early_stopping_rounds=40, verbose_eval=20)

[0]	train-rmse:0.979616	valid-rmse:0.957112
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 40 rounds.
[20]	train-rmse:0.202263	valid-rmse:0.253519
[40]	train-rmse:0.147433	valid-rmse:0.295611
[60]	train-rmse:0.136245	valid-rmse:0.363534
Stopping. Best iteration:
[23]	train-rmse:0.183221	valid-rmse:0.251212



In [22]:
bstpred1 = bst1.predict(dtest)
bstpred1 = np.exp(bstpred1)-1

In [23]:
bst2 = xgb.train(xgb_params2, dtrain2, 1000, watchlist2, early_stopping_rounds=40, verbose_eval=20)

[0]	train-rmse:4.91088	valid-rmse:4.51773
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 40 rounds.
[20]	train-rmse:0.719407	valid-rmse:0.624172
[40]	train-rmse:0.376411	valid-rmse:0.701333
[60]	train-rmse:0.343445	valid-rmse:0.756637
Stopping. Best iteration:
[24]	train-rmse:0.556766	valid-rmse:0.598527



In [24]:
bstpred2 = bst2.predict(dtest)
bstpred2 = np.exp(bstpred2)-1

In [25]:
subpy6 = pd.DataFrame({'ID':test['ID'], 'Price':bstpred1, 'Number_Of_Sales':bstpred2})
subpy6.to_csv("subpy6.csv", index=False) # 0.71

### trying time series cross validation

In [32]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

In [28]:
from sklearn.linear_model import Ridge,RidgeCV
aps = [0.001,0.01,0.1,0.5,0.3,0.05,2,5,7,8,10,15,20,30,40]

In [29]:
r1 = RidgeCV(alphas=aps)

In [30]:
r1.fit(train_X[cols_to_use], np.log1p(train_X['Price']))

RidgeCV(alphas=[0.001, 0.01, 0.1, 0.5, 0.3, 0.05, 2, 5, 7, 8, 10, 15, 20, 30, 40],
    cv=None, fit_intercept=True, gcv_mode=None, normalize=False,
    scoring=None, store_cv_values=False)

In [31]:
r1.alpha_

2.0

### Price

In [33]:
r1tr = Ridge(alpha=2)
r1tr.fit(train_X[cols_to_use], np.log1p(train_X['Price']))
r1pred = r1tr.predict(valid_X[cols_to_use])
r1pred = np.exp(r1pred)-1
np.sqrt(mean_squared_error(valid_X['Price'], r1pred))

78.561621222499667

In [34]:
r1tr = Ridge(alpha=2)
r1tr.fit(train[cols_to_use], np.log1p(train['Price']))
r1pred = r1tr.predict(test[cols_to_use])
r1pred = np.exp(r1pred)-1

### Number Of Sales

In [36]:
r1.fit(train_X[cols_to_use], np.log1p(train_X['Number_Of_Sales']))
r1.alpha_

0.050000000000000003

In [38]:
r1tr = Ridge(alpha=r1.alpha_)
r1tr.fit(train_X[cols_to_use], np.log1p(train_X['Number_Of_Sales']))
r2pred = r1tr.predict(valid_X[cols_to_use])
r2pred = np.exp(r2pred)-1
np.sqrt(mean_squared_error(valid_X['Price'], r2pred))

21084.854336772052

In [39]:
r1tr = Ridge(alpha=r1.alpha_)
r1tr.fit(train[cols_to_use], np.log1p(train['Number_Of_Sales']))
r2pred = r1tr.predict(test[cols_to_use])
r2pred = np.exp(r2pred)-1

In [40]:
subpy7 = pd.DataFrame({'ID':test['ID'], 'Price': r1pred, 'Number_Of_Sales': r2pred})
subpy7.to_csv("subpy7.csv", index=False)

### Ensemble  - Ridge + XGB

In [49]:
price_med = (bstpred1 + r1pred)/2
sales_med = (bstpred2 + r2pred) /2

subpy8 = pd.DataFrame({'ID':test.ID, 'Price': price_med, 'Number_Of_Sales':sales_med})

In [50]:
subpy8.head()

,ID,Number_Of_Sales,Price
0,30413_20160701,5.435264,298.529892
1,30413_20160702,5.531651,298.488183
2,30413_20160703,5.630794,297.922312
3,30413_20160704,4.824356,339.711178
4,30413_20160705,4.906677,339.662137


In [51]:
subpy8.to_csv("subpy8.csv",index=False) #3.something

### trying xgboost with one hot encoded Category 2 and Category 1

In [88]:
cat1 = pd.get_dummies(test['Category_1'],prefix='cat1_')

In [89]:
sumlist = cat1.sum()
sumlist = sumlist[sumlist > 1000]
keepcols = sumlist.index.values

In [90]:
print cat1.shape

(266248, 327)


In [91]:
cat1 = cat1[keepcols]

In [92]:
print cat1.shape

(266248, 66)


In [93]:
cat2 = pd.get_dummies(train['Category_1'],prefix='cat1_')

In [94]:
print cat2.shape

(881871, 359)


In [95]:
cat2 = cat2[keepcols]

In [96]:
train = pd.concat([train, cat2], axis=1)
test = pd.concat([test,cat1], axis=1)

In [97]:
print train.shape
print test.shape

(881871, 81)
(266248, 79)


In [99]:
pd.get_option("display.max_columns",1000)

20

In [104]:
pad1 = pd.get_dummies(test['Category_2'],prefix = 'cat2_')
pad2 = pd.get_dummies(train['Category_2'],prefix='cat2_')

In [108]:
train = pd.concat([train, pad2], axis=1)
test = pd.concat([test, pad1], axis=1)

In [109]:
train.drop(['Category_1','Category_2'],axis=1,inplace=True)
test.drop(['Category_1','Category_2'],axis=1,inplace=True)

### XGBoost Again

In [112]:
train.columns

Index([u'ID', u'Item_ID', u'Datetime', u'Category_3', u'Price',
       u'Number_Of_Sales', u'item_count', u'dayofweek', u'weekyear',
       u'dayofmonth', u'month', u'weekYearIDPrice', u'weekYearIDSales',
       u'cat1__0', u'cat1__17', u'cat1__27', u'cat1__35', u'cat1__40',
       u'cat1__46', u'cat1__53', u'cat1__56', u'cat1__66', u'cat1__79',
       u'cat1__83', u'cat1__90', u'cat1__93', u'cat1__110', u'cat1__114',
       u'cat1__117', u'cat1__132', u'cat1__134', u'cat1__137', u'cat1__155',
       u'cat1__160', u'cat1__166', u'cat1__173', u'cat1__176', u'cat1__183',
       u'cat1__187', u'cat1__188', u'cat1__190', u'cat1__197', u'cat1__206',
       u'cat1__222', u'cat1__229', u'cat1__233', u'cat1__235', u'cat1__236',
       u'cat1__239', u'cat1__250', u'cat1__251', u'cat1__255', u'cat1__263',
       u'cat1__271', u'cat1__274', u'cat1__278', u'cat1__285', u'cat1__294',
       u'cat1__297', u'cat1__298', u'cat1__299', u'cat1__307', u'cat1__308',
       u'cat1__310', u'cat1__313', u'ca

In [119]:
cols_to_drop = ['ID','Datetime','Category_1','Category_2','Number_Of_Sales','Price']
cols_to_use = set(train.columns) - set(cols_to_drop)
cols_to_use = list(cols_to_use)

In [110]:
train_X = train.loc[(train['Datetime'] < "2016-01-01") & (train['Datetime'] >= "2014-01-01")]
valid_X = train.loc[train['Datetime'] >= '2016-01-01']

In [17]:
import xgboost as xgb

/home/manish/anaconda2/envs/package15/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [120]:
dtrain1 = xgb.DMatrix(data=train_X[cols_to_use], label = np.log1p(train_X['Price']))
dvalid1 = xgb.DMatrix(data=valid_X[cols_to_use], label = np.log1p(valid_X['Price']))

dtrain2 = xgb.DMatrix(data=train_X[cols_to_use], label = np.log1p(train_X['Number_Of_Sales']))
dvalid2 = xgb.DMatrix(data=valid_X[cols_to_use], label = np.log1p(valid_X['Number_Of_Sales']))

In [121]:
dtest = xgb.DMatrix(data=test[cols_to_use])

In [122]:
xgb_params1 = {
    'colsample_bytree': 1,
    'subsample': 1,
    'eta': 0.1,
    'objective': 'reg:linear',
    'max_depth': 6,
    'min_child_weight': 10,
    'eval_metric': 'rmse',
    'alpha':8
}

xgb_params2 = {
    'colsample_bytree': 1,
    'subsample': 1,
    'eta': 0.1,
    'objective': 'reg:linear',
    'max_depth': 6,
    'min_child_weight': 0,
    'eval_metric': 'rmse',
    'alpha':8
}
watchlist1  = [ (dtrain1,'train'),(dvalid1,'valid')]
watchlist2  = [ (dtrain2,'train'),(dvalid2,'valid')]


In [123]:
bst1 = xgb.train(xgb_params1, dtrain1, 1000, watchlist1, early_stopping_rounds=40, verbose_eval=20)

[0]	train-rmse:0.97562	valid-rmse:0.956928
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 40 rounds.
[20]	train-rmse:0.232811	valid-rmse:0.256245
[40]	train-rmse:0.194149	valid-rmse:0.284187
[60]	train-rmse:0.189333	valid-rmse:0.335512
Stopping. Best iteration:
[23]	train-rmse:0.2173	valid-rmse:0.253602



In [124]:
bstpred1 = bst1.predict(dtest)
bstpred1 = np.exp(bstpred1)-1

In [125]:
bst2 = xgb.train(xgb_params2, dtrain2, 1000, watchlist2, early_stopping_rounds=40, verbose_eval=20)

[0]	train-rmse:4.74344	valid-rmse:4.53928
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 40 rounds.
[20]	train-rmse:0.838146	valid-rmse:0.74287
[40]	train-rmse:0.596959	valid-rmse:0.710569
[60]	train-rmse:0.580999	valid-rmse:0.760991
Stopping. Best iteration:
[27]	train-rmse:0.663505	valid-rmse:0.664155



In [126]:
bstpred2 = bst2.predict(dtest)
bstpred2 = np.exp(bstpred2)-1

In [127]:
subpy9 = pd.DataFrame({'ID':test['ID'], 'Price':bstpred1, 'Number_Of_Sales':bstpred2})
subpy9.to_csv("subpy9.csv", index=False) #0.92

### lag features try

In [172]:
train = pd.read_csv('train.csv',parse_dates=['Datetime'])
test = pd.read_csv('test.csv',parse_dates=['Datetime'])

In [173]:
max_date = test['Datetime'].max()
min_date = train['Datetime'].min()
days = (max_date - min_date).days + 1

In [174]:
import datetime

In [175]:
#Master list of dates
dates = [min_date + datetime.timedelta(days=x) for x in range(days)]

In [176]:
# Master list of Items
unique_items = list(set(train['Item_ID'].unique()) | set(test['Item_ID'].unique()))
unique_items.sort()
num_unique_items = len(unique_items)

In [177]:
date_index = np.repeat(dates, num_unique_items)

In [178]:
item_index = np.concatenate([unique_items]* days)

In [179]:
row_count = days * len(unique_items)

In [180]:
train_test_master = pd.DataFrame(index=range(row_count), columns=train.columns)

In [181]:
train_test_master['Datetime'] = date_index
train_test_master['Item_ID'] = item_index

In [182]:
train_test_master.set_index(['Datetime', 'Item_ID'], drop=True, inplace=True)
train.set_index(['Datetime', 'Item_ID'], drop=True, inplace=True)
test.set_index(['Datetime', 'Item_ID'], drop=True, inplace=True)

In [183]:
# Update the master index with train and test
train_test_master.update(train)
train_test_master.update(test)

In [165]:
train_test_master

ID Category_3 Category_2 Category_1  Price  \
Datetime   Item_ID                                                           
2014-01-01 29654               NaN        NaN        NaN        NaN    NaN   
           29655               NaN        NaN        NaN        NaN    NaN   
           29656               NaN        NaN        NaN        NaN    NaN   
           29657               NaN        NaN        NaN        NaN    NaN   
           29658               NaN        NaN        NaN        NaN    NaN   
           29661    29661_20140101          0          2        379  0.122   
           29662    29662_20140101          0        NaN          0  1.649   
           29664               NaN        NaN        NaN        NaN    NaN   
           29666               NaN        NaN        NaN        NaN    NaN   
           29667    29667_20140101          0          2        221  1.256   
           29668               NaN        NaN        NaN        NaN    NaN   
           29669               NaN        NaN        NaN        NaN    NaN   
           29670               NaN        NaN        NaN        NaN    NaN   
           29671    29671_20140101          0          1        236    2.1   
           29672               NaN        NaN        NaN        NaN    NaN   
           29673               NaN        NaN        NaN        NaN    NaN   
           29674               NaN        NaN        NaN        NaN    NaN   
           29675               NaN        NaN        NaN        NaN    NaN   
           29676               NaN        NaN        NaN        NaN    NaN   
           29677               NaN        NaN        NaN        NaN    NaN   
           29678               NaN        NaN        NaN        NaN    NaN   
           29679               NaN        NaN        NaN        NaN    NaN   
           29683    29683_20140101          0          2        310   0.03   
           29684               NaN        NaN        NaN        NaN    NaN   
           29685               NaN        NaN        NaN        NaN    NaN   
           29686    29686_20140101          0          4        382  0.172   
           29687               NaN        NaN        NaN        NaN    NaN   
           29688               NaN        NaN        NaN        NaN    NaN   
           29689               NaN        NaN        NaN        NaN    NaN   
           29691               NaN        NaN        NaN        NaN    NaN   
...                            ...        ...        ...        ...    ...   
2016-12-31 31345    31345_20161231          1          1        296    NaN   
           31346    31346_20161231          1          1        349    NaN   
           31347    31347_20161231          0          2         35    NaN   
           31348    31348_20161231          0        NaN          0    NaN   
           31349    31349_20161231          0          3        173    NaN   
           31351    31351_20161231          0          2        123    NaN   
           31352    31352_20161231          0          5        257    NaN   
           31353    31353_20161231          0          5        127    NaN   
           31354    31354_20161231          0          5        114    NaN   
           31355    31355_20161231          0          4         93    NaN   
           31356               NaN        NaN        NaN        NaN    NaN   
           31357    31357_20161231          0          2        288    NaN   
           31358    31358_20161231          0          5        338    NaN   
           31359    31359_20161231          1          4        229    NaN   
           31360    31360_20161231          0        NaN          0    NaN   
           31361    31361_20161231          1          3        114    NaN   
           31362    31362_20161231          1          3        117    NaN   
           31363    31363_20161231          0          5        388    NaN   
           31364    31364_20161231          0         

In [184]:
# 1 day lag
train_test_master['lag_1'] = train_test_master['Sales'].shift(-1*NUM_STORES).fillna(NULL_VALUE)

KeyError: 'Sales'